In [1]:
import os, glob, numpy as np
import subprocess
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import pandas as pd
import mlflow
import mlflow.pytorch
import psutil
from pynvml import (
    nvmlInit, nvmlDeviceGetHandleByIndex,
    nvmlDeviceGetUtilizationRates, nvmlDeviceGetMemoryInfo,
    nvmlDeviceGetTemperature, NVML_TEMPERATURE_GPU
)

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

# MLFlow setup
mlflow.set_experiment("PannsMLP")
try: 
    mlflow.end_run() # end pre-existing run, if there was one
except:
    pass
finally:
    mlflow.start_run(log_system_metrics=True) # Start MLFlow run
gpu_info = next(
    (subprocess.run(cmd, capture_output=True, text=True).stdout for cmd in ["nvidia-smi", "rocm-smi"] 
     if subprocess.run(f"command -v {cmd}", shell=True, capture_output=True).returncode == 0),
    "No GPU found."
)
mlflow.log_text(gpu_info, "gpu-info.txt")

2025/05/01 00:13:15 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


Using device: cuda


In [5]:
EMB_DIR = '/mnt/birdclef/processed/embeddings'

meta = pd.read_csv('/mnt/birdclef/raw/train.csv')
label2idx = {lab:i for i, lab in enumerate(sorted(meta['primary_label'].unique()))}
num_classes = len(label2idx)

# 1) Gather embedding files
print('Loading Paths')
all_paths  = sorted(glob.glob(os.path.join(EMB_DIR, '**', '*_emb.npz'), recursive=True))
print('Loading Labels')
all_labels = [int(np.load(p)['label']) for p in tqdm(all_paths, desc="Loading Labels")]
# 2) Split into train/test, try stratify then fallback
try:
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_paths, all_labels,
        test_size=0.2,
        random_state=42
    )
except ValueError:
    print("Warning: stratify failed (too few samples in some classes), splitting without stratify.")
    train_paths, test_paths, train_labels, test_labels = train_test_split(
        all_paths, all_labels,
        test_size=0.2,
        shuffle=True,
        random_state=42
    )

# 3) Dataset definition
class EmbeddingDataset(Dataset):
    def __init__(self, paths):
        self.paths = paths
    def __len__(self):
        return len(self.paths)
    def __getitem__(self, idx):
        data = np.load(self.paths[idx])
        emb  = data['embedding'].astype(np.float32)
        lbl  = int(data['label'])
        return torch.from_numpy(emb), torch.tensor(lbl)

# 4) Instantiate & wrap in DataLoaders
train_ds = EmbeddingDataset(train_paths)
test_ds  = EmbeddingDataset(test_paths)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True,  num_workers=8, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

print(f"Train samples: {len(train_ds)}, Test samples: {len(test_ds)}")

Loading Paths
Loading Labels


Loading Labels:   0%|          | 0/11236 [00:00<?, ?it/s]

Train samples: 8988, Test samples: 2248


In [6]:
nvmlInit()
gpu_handle = nvmlDeviceGetHandleByIndex(0)

def log_system_metrics_mlflow(step=None):
    # CPU Utilization (%)
    cpu_util = psutil.cpu_percent()
    mlflow.log_metric("system.cpu.utilization", cpu_util, step=step)

    # Memory Usage (bytes)
    mem = psutil.virtual_memory()
    mlflow.log_metric("system.memory.used", mem.used, step=step)
    mlflow.log_metric("system.memory.percent", mem.percent, step=step)

    # GPU Utilization (%)
    gpu_util = nvmlDeviceGetUtilizationRates(gpu_handle).gpu
    mlflow.log_metric("system.gpu.0.utilization", gpu_util, step=step)

    # GPU Memory (bytes)
    gpu_mem = nvmlDeviceGetMemoryInfo(gpu_handle)
    mlflow.log_metric("system.gpu.0.memory.used", gpu_mem.used, step=step)
    mlflow.log_metric("system.gpu.0.memory.percent", (gpu_mem.used / gpu_mem.total) * 100, step=step)

    # GPU Temperature (°C)
    gpu_temp = nvmlDeviceGetTemperature(gpu_handle, NVML_TEMPERATURE_GPU)
    mlflow.log_metric("system.gpu.0.temperature", gpu_temp, step=step)

In [7]:
# Cell 3 — Define MLP, criterion, optimizer & count params
class MLPClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout=0.5):
        super().__init__()
        layers = []
        dims = [input_dim] + hidden_dims
        for i in range(len(hidden_dims)):
            layers += [
                nn.Linear(dims[i], dims[i+1]),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout)
            ]
        layers.append(nn.Linear(dims[-1], num_classes))
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

sample_emb, sample_lbl = next(iter(train_loader))
input_dim   = sample_emb.shape[1]

model     = MLPClassifier(input_dim, [1024, 512], num_classes, dropout=0.5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {total_params:,}")


Total trainable parameters: 2,728,654


In [8]:
# Log hyperparameters
mlflow.log_params({
    "input_dim": input_dim,
    "hidden_dims": [1024, 512],
    "dropout": 0.5,
    "batch_size": 32,
    "lr": 1e-3,
    "num_epochs": 20
})

In [10]:
# Training loop with MLFlow logging
num_epochs = 5
best_acc   = 0.0
checkpoint_path = 'best_panns_mlp_checkpoint.pth'

for epoch in range(1, num_epochs+1):
    model.train()
    train_loss = 0.0
    train_correct = 0
    total = 0
    for emb, lbl in tqdm(train_loader, desc=f"Epoch {epoch} ▶ Train"):
        emb, lbl = emb.to(device), lbl.to(device)
        optimizer.zero_grad()
        logits = model(emb)
        loss   = criterion(logits, lbl)
        loss.backward()
        optimizer.step()

        train_loss    += loss.item() * emb.size(0)
        train_correct += (logits.argmax(dim=1) == lbl).sum().item()
        total         += emb.size(0)

    train_loss /= total
    train_acc   = train_correct / total

    model.eval()
    val_loss, val_correct, val_total = 0.0, 0, 0
    with torch.no_grad():
        for emb, lbl in tqdm(test_loader, desc=f"Epoch {epoch} ✅ Val"):
            emb, lbl = emb.to(device), lbl.to(device)
            logits = model(emb)
            loss   = criterion(logits, lbl)

            val_loss    += loss.item() * emb.size(0)
            val_correct += (logits.argmax(dim=1) == lbl).sum().item()
            val_total   += emb.size(0)

    val_loss /= val_total
    val_acc   = val_correct / val_total

    mlflow.log_metrics({
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc
    }, step=epoch)
    log_system_metrics_mlflow(step=epoch)
    
    print(f"\nEpoch {epoch:02d} | Train: loss={train_loss:.4f}, acc={train_acc:.4f} | Val: loss={val_loss:.4f}, acc={val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict':       model.state_dict(),
            'optimizer_state_dict':   optimizer.state_dict(),
            'best_validation_acc':    best_acc,
            'input_dim':              input_dim,
            'hidden_dims':            [1024, 512],
            'num_classes':            num_classes
        }, checkpoint_path)
        print(f"✔️  New best model saved (epoch {epoch}, val_acc={val_acc:.4f})")
        mlflow.log_artifact(checkpoint_path, artifact_path="model")

mlflow.log_metric("best_val_accuracy", best_acc)
mlflow.end_run()

print(f"\n🎉 Best validation accuracy: {best_acc:.4f}")

Epoch 1 ▶ Train:   0%|          | 0/281 [00:00<?, ?it/s]

Epoch 1 ✅ Val:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 01 | Train: loss=4.3306, acc=0.0919 | Val: loss=3.9494, acc=0.1361
✔️  New best model saved (epoch 1, val_acc=0.1361)


Epoch 2 ▶ Train:   0%|          | 0/281 [00:00<?, ?it/s]

Epoch 2 ✅ Val:   0%|          | 0/71 [00:00<?, ?it/s]

...

Epoch 5 ▶ Train:   0%|          | 0/281 [00:00<?, ?it/s]

Epoch 5 ✅ Val:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 05 | Train: loss=3.1939, acc=0.2621 | Val: loss=3.0008, acc=0.3123
✔️  New best model saved (epoch 5, val_acc=0.3123)


2025/05/01 00:46:51 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2025/05/01 00:46:51 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


🏃 View run grandiose-squid-227 at: http://192.5.86.175:8000/#/experiments/1/runs/cfa080bf21d442eeb32dd4c042f26c64
🧪 View experiment at: http://192.5.86.175:8000/#/experiments/1

🎉 Best validation accuracy: 0.3123


In [32]:
# Cell 5 — (Optional) Load best checkpoint for inference or continued training
ckpt = torch.load('best_panns_mlp_checkpoint.pt', map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
optimizer.load_state_dict(ckpt['optimizer_state_dict'])
print(f"Loaded checkpoint from epoch {ckpt['epoch']} with val_acc={ckpt['best_validation_acc']:.4f}")


Loaded checkpoint from epoch 13 with val_acc=0.5565


/tmp/ipykernel_8068/2758929225.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('best_panns_mlp_checkpoint.pt', map_location=device)
